In [ ]:
print ("Start 10/08/2025")

: 

In [ ]:
# ============================================
# 📦 Core Imports
# ============================================
import os
import pandas as pd
from dotenv import load_dotenv

# ============================================
# 🧠 LLM & Embeddings
# ============================================
import ollama
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_groq import ChatGroq

# ============================================
# 📄 Document Processing
# ============================================
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# ============================================
# 📑 File Conversion
# ============================================
import pypandoc

# ============================================
# 🔐 Environment Setup
# ============================================
# Load environment variables (ensure API keys are stored in a .env file)
load_dotenv()


: 

In [ ]:


# Use the llama model (change to llama2, llama3 if installed)
llm = Ollama(model="llama3.3")

response = llm.invoke("What is the capital of France?")
print(response)

OllamaEndpointNotFoundError: Ollama call failed with status code 404. Maybe your model is not found and you should pull the model with `ollama pull llama3.3`.